<a href="https://colab.research.google.com/github/SonalDhoke/Programming_with_Data_Analysis_PY/blob/main/st20336540_CMP7005_PDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

---



**Overview**

Apart from land and water, air is the most important resource for life.Thus Air pollution remains one of the most persistent environmental challenges in India and monitoring Air Quality Index (AQI) trends is very essential for understanding its impact on public health and urban sustainability.

With various technological advancements, a vast amount of data on ambient air quality is generated and used to establish the quality of air in different areas.

**What is Air Quality Index and why it's needed ?**

Air quality can be described by measuring the levels of different pollutants in the air. But as the number of pollutants and monitoring stations increases, the information becomes difficult to understand—even for scientists. For the general public, looking at raw pollution data, graphs, or detailed reports can be confusing and unhelpful. Because of this, people may lose interest and not fully understand how good or bad the air quality is, or what actions the government is taking to control pollution.

However, it is important for everyone, especially people with health issues, to know the daily air pollution levels in a simple and clear way. A well-informed public is also more likely to support actions and policies that help improve air quality.

To make air quality easy to understand, many countries developed the Air Quality Index (AQI). AQI turns complex pollution measurements into a single number and category (like “Good,” “Moderate,” or “Poor”), making it easier for people to quickly understand the state of the air and take necessary precautions. This system has been used effectively around the world for more than 30 years.

**What this Dataset is about ?**

This dataset help us to analyze and understand the patterns,the problems related to Air Quality, how it is measured and its impact.This dataset examines a multy-city AQI spanning from January 2015 to Juky 2020, providing a five and a half year view of air quality conditions across india. The dataset captures daily AQI observations from 28 major cities.
By analyzing this dataset, we are aiming to identify temporal trends, seasonal behaviors, and spatial differences in air quality across India.Through programming and data analytics tools, this project transforms raw AQI data into meaningful insights that can support environmental assessment and policy discussions.

**Importing Required Libraries for Analysis**

In [155]:
# importing the necessary libraries for data analysis
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

**Connecting to Google Drive**

In [156]:
from google.colab import drive

In [157]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [158]:
drive_path ='/content/drive/MyDrive/Colab/Programming_DA/PDA_Assessment/Assessment Data-20251118'

**Combining all the CSV files in the drive path:**

All the files in the folder are checked one by one, and the ones that are with CSV extension that files are selected. Each CSV file is loaded into a Pandas DataFrame and stored in a list. After loading all the files, the DataFrames in the list are joined together using pd.concat() to create one complete DataFrame. This gives us a single dataset that contains all the data from the individual CSV files.

In [159]:
dataframes = []
for filename in os.listdir(drive_path):
    if filename.endswith('.csv'):  # Check if the file is a CSV file
        file_path = os.path.join(drive_path, filename)
        df = pd.read_csv(file_path)  # Read the CSV file into a DataFrame
        dataframes.append(df)  # Add the DataFrame to the list

In [160]:
df = pd.concat(dataframes, ignore_index=True)
df

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Brajrajnagar,07/12/2017,171.38,311.68,NaN,41.07,NaN,7.78,3.26,5.83,4.69,0.00,NaN,NaN,NaN,NaN
1,Brajrajnagar,08/12/2017,126.67,192.81,NaN,20.16,NaN,7.97,2.94,5.43,10.51,0.00,NaN,NaN,304.0,Very Poor
2,Brajrajnagar,09/12/2017,105.11,154.39,NaN,18.40,NaN,8.15,2.83,5.10,10.33,0.00,NaN,NaN,291.0,Poor
3,Brajrajnagar,10/12/2017,65.25,99.73,NaN,15.21,NaN,7.90,2.74,4.75,8.37,0.00,NaN,NaN,189.0,Moderate
4,Brajrajnagar,11/12/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29526,Ahmedabad,27/06/2020,62.12,118.67,9.18,56.35,19.86,NaN,0.49,12.44,68.05,1.32,37.76,1.62,92.0,Satisfactory
29527,Ahmedabad,28/06/2020,31.57,NaN,6.37,23.99,16.40,NaN,0.52,11.01,26.34,1.37,49.58,1.34,82.0,Satisfactory
29528,Ahmedabad,29/06/2020,29.75,127.98,9.06,25.15,18.92,NaN,0.67,12.10,34.99,1.39,60.21,0.79,74.0,Satisfactory
29529,Ahmedabad,30/06/2020,40.02,121.10,7.09,58.92,33.41,NaN,0.73,16.39,41.64,1.21,44.10,1.35,98.0,Satisfactory


**Basic Data Understanding:**

To understand my dataset,I've checked the columns and their data types. Also reviewed the descriptive statistics and identified ant missing values.I've used a heatmap to visualize where the missing data occurs,making it easier to see the patterns.This information will guide the preprocessing steps later on.


In [161]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [162]:
df.head()

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
0,Brajrajnagar,07/12/2017,171.38,311.68,NaN,41.07,NaN,7.78,3.26,5.83,4.69,0.0,NaN,NaN,NaN,NaN
1,Brajrajnagar,08/12/2017,126.67,192.81,NaN,20.16,NaN,7.97,2.94,5.43,10.51,0.0,NaN,NaN,304.0,Very Poor
2,Brajrajnagar,09/12/2017,105.11,154.39,NaN,18.40,NaN,8.15,2.83,5.10,10.33,0.0,NaN,NaN,291.0,Poor
3,Brajrajnagar,10/12/2017,65.25,99.73,NaN,15.21,NaN,7.90,2.74,4.75,8.37,0.0,NaN,NaN,189.0,Moderate
4,Brajrajnagar,11/12/2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
df.columns

Index(['City', 'Date', 'PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2',
       'O3', 'Benzene', 'Toluene', 'Xylene', 'AQI', 'AQI_Bucket'],
      dtype='object')

**Description of dataset columns:**


* City – Name of the city where the air quality was measured
* Date – The specific day on which the readings were recorded.
* PM2.5 – Fine particulate matter smaller than 2.5 micrometers, harmful as it can enter the lungs.
* PM10 – Particulate matter smaller than 10 micrometers, mainly dust and smoke.
* NO – Nitric oxide, a gas produced mainly from vehicle and industrial emissions.
* NO2 – Nitrogen dioxide, a toxic gas that contributes to smog and respiratory problems.
* NOx – A group of nitrogen oxides (NO + NO2) commonly used to track pollution from combustion sources.
* NH3 – Ammonia, mainly released from agriculture and waste, contributing to particulate pollution.
* CO – Carbon monoxide, a colorless gas from incomplete combustion, harmful when inhaled.
* SO2 – Sulfur dioxide, a gas from burning fossil fuels, causing irritation and acid rain.
* O3 – Ozone at ground level, a harmful pollutant formed by chemical reactions in sunlight.
* Benzene – A toxic volatile organic compound (VOC) often released from vehicles and industries.
* Toluene – A VOC found in solvents and fuels, contributing to air pollution.
* Xylene – Another VOC from industrial activity and traffic emissions.
* AQI – Air Quality Index, a single value that shows how clean or polluted the air is.
* AQI_Bucket – Category describing air quality (e.g., Good, Moderate, Poor) based on the AQI value



This structure is consistent across the files, with each file representing data from a different monitoring station.

In [164]:
df.tail(5)

,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
29526,Ahmedabad,27/06/2020,62.12,118.67,9.18,56.35,19.86,NaN,0.49,12.44,68.05,1.32,37.76,1.62,92.0,Satisfactory
29527,Ahmedabad,28/06/2020,31.57,NaN,6.37,23.99,16.40,NaN,0.52,11.01,26.34,1.37,49.58,1.34,82.0,Satisfactory
29528,Ahmedabad,29/06/2020,29.75,127.98,9.06,25.15,18.92,NaN,0.67,12.10,34.99,1.39,60.21,0.79,74.0,Satisfactory
29529,Ahmedabad,30/06/2020,40.02,121.10,7.09,58.92,33.41,NaN,0.73,16.39,41.64,1.21,44.10,1.35,98.0,Satisfactory
29530,Ahmedabad,01/07/2020,37.63,NaN,4.42,35.04,20.17,NaN,0.28,14.40,9.69,1.73,47.05,1.87,119.0,Moderate


In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29531 entries, 0 to 29530
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City        29531 non-null  object 
 1   Date        29531 non-null  object 
 2   PM2.5       24933 non-null  float64
 3   PM10        18391 non-null  float64
 4   NO          25949 non-null  float64
 5   NO2         25946 non-null  float64
 6   NOx         25346 non-null  float64
 7   NH3         19203 non-null  float64
 8   CO          27472 non-null  float64
 9   SO2         25677 non-null  float64
 10  O3          25509 non-null  float64
 11  Benzene     23908 non-null  float64
 12  Toluene     21490 non-null  float64
 13  Xylene      11422 non-null  float64
 14  AQI         24850 non-null  float64
 15  AQI_Bucket  24850 non-null  object 
dtypes: float64(13), object(3)
memory usage: 3.6+ MB


In [166]:
df.shape

(29531, 16)

# **Inference :**


---


The dataset contains 29,531 records with 16 columns, and while the City and Date columns are fully complete, several pollutant measurements show missing data. Key pollutants such as PM2.5, PM10, NH3, Toluene, and Xylene have a significant number of missing values, with Xylene having the least completeness. Other pollutants, including NO, NO2, NOx, SO2, O3, CO, and Benzene, also show moderate gaps but are more complete in comparison. The AQI and AQI_Bucket columns have some missing entries as well, which may affect overall air quality classification. Overall, the dataset is usable but will require proper handling of missing values—such as imputation or filtering—before performing deeper analysis.

In [167]:
City = df['City'].value_counts()
print(f'Total number of cities in the dataset : {len(City)}')
City

Total number of cities in the dataset : 26


,count
City,
Lucknow,2009
Chennai,2009
Bengaluru,2009
Mumbai,2009
Delhi,2009
Ahmedabad,2009
Hyderabad,2006
Patna,1858
Gurugram,1679


In [168]:
df['City'].unique()

array(['Brajrajnagar', 'Visakhapatnam', 'Thiruvananthapuram', 'Gurugram',
       'Kochi', 'Lucknow', 'Amritsar', 'Coimbatore', 'Ernakulam',
       'Amaravati', 'Chennai', 'Kolkata', 'Shillong', 'Delhi',
       'Bengaluru', 'Aizawl', 'Patna', 'Mumbai', 'Jorapokhar',
       'Chandigarh', 'Jaipur', 'Hyderabad', 'Talcher', 'Guwahati',
       'Bhopal', 'Ahmedabad'], dtype=object)

In [169]:
print(f"There are  {df.duplicated().sum()} duplicates in the dataset")

There are  0 duplicates in the dataset


# **Inference:**


---

***Data Understanding***
*  **Dataset Structure** : The dataset contains 29,531 rows and 16 columns.It includes pollutant measurements, city names, dates, AQI values, and AQI categories.

   There are mainly two types of columns,i.e. Categorical and Numerical.

   *Categorical columns:*  City, Date, AQI_Bucket.(3 Columns)

   *Numerical columns:*   All pollutant concentrations and AQI.(13 columns)

* **Data Completeness** : The column City and Date have no missing values.
The pollutants show varying levels of missing data:

    *Least missing:* CO, NO, NO2, NOx, SO2, O3, Benzene

    *Moderate missing:* PM2.5, PM10, NH3, Toluene,  AQI, AQI_Bucket

    *Most missing:* Xylene


**Displaying the percentage of missing value**

In [170]:
#checking Missing values
def missing_values_table(df):
    # Total missing values
    mis_val = df.isnull().sum()

    # Percentage of missing values
    mis_val_percent = 100 * mis_val / len(df)

    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)

    mis_val_table = mis_val_table.rename(columns={0: 'Missing Values', 1: '% of Total Values'})

    # Sort the table by percentage of missing descending
    mis_val_table = mis_val_table.sort_values('% of Total Values', ascending=False)

    return mis_val_table

missing_values = missing_values_table(df)
display(missing_values.style.background_gradient(cmap='Greens'))


,Missing Values,% of Total Values
Xylene,18109,61.322001
PM10,11140,37.723071
NH3,10328,34.973418
Toluene,8041,27.229014
Benzene,5623,19.041008
AQI,4681,15.851139
AQI_Bucket,4681,15.851139
PM2.5,4598,15.570079
NOx,4185,14.171549
O3,4022,13.619586


In [171]:
missing_values_per_city = df.groupby('City').apply(lambda x: x.isnull().sum())
missing_values_per_city

/tmp/ipython-input-410150535.py:1: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,City,Date,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3,Benzene,Toluene,Xylene,AQI,AQI_Bucket
City,,,,,,,,,,,,,,,,
Ahmedabad,0,0,628,1602,609,608,594,2009,609,640,685,483,483,484,675,675
Aizawl,0,0,2,1,0,0,0,0,0,0,9,15,15,113,2,2
Amaravati,0,0,59,55,55,55,55,55,97,65,55,75,75,269,110,110
Amritsar,0,0,128,63,93,44,289,42,76,168,115,195,212,213,95,95
Bengaluru,0,0,146,360,6,6,4,203,11,6,144,266,93,2009,99,99
Bhopal,0,0,9,9,9,9,9,9,9,9,9,289,289,289,11,11
Brajrajnagar,0,0,185,173,243,234,175,226,150,193,203,173,938,938,225,225
Chandigarh,0,0,15,0,4,4,4,10,0,0,0,0,0,0,5,5
Chennai,0,0,117,1707,36,36,36,462,25,36,49,285,253,2009,125,125


# **Inference:**


---

The table shows that missing data is unevenly distributed across cities and pollutants.  
* The City and Date columns have no missing values—ensuring complete spatial and temporal coverage—the pollutant measurements vary greatly in completeness across locations.
* Highly inconsistent pollutants such as **Xylene, PM10, NH3, Toluene, and Benzene** show large gaps in many cities, especially in places like Bengaluru, Chennai, Jorapokhar, Lucknow, and Mumbai, suggesting irregular monitoring, sensor downtime, or missing data transmission during certain periods.
* In contrast, gases like  **NO, NO2, NOx, SO2, and CO** exhibit relatively fewer missing entries across most cities, indicating more stable measurement practices for these pollutants.
* **AQI and AQI_Bucket** also show missingness patterns that correspond directly with gaps in pollutant values, since they rely on these measurements for calculation.
* Overall, the dataset provides strong temporal coverage across all cities, but pollutant monitoring is inconsistent—particularly for volatile organic compounds (VOC) i.e., Benzene,Toluene,Xylene and particulate matter—which will require careful cleaning or imputation before performing reliable air quality comparisons or trend analysis.

In [172]:
print("Total missing values per Cities:")
print(df.groupby('City').apply(lambda x: x.isnull().sum().sum()))

Total missing values per Cities:
City
Ahmedabad             10784
Aizawl                  159
Amaravati              1190
Amritsar               1828
Bengaluru              3452
Bhopal                  970
Brajrajnagar           4281
Chandigarh               47
Chennai                5301
Coimbatore              693
Delhi                  1087
Ernakulam               275
Gurugram               6843
Guwahati               1033
Hyderabad              1583
Jaipur                 1416
Jorapokhar             8603
Kochi                   356
Kolkata                 816
Lucknow                6082
Mumbai                14201
Patna                  6214
Shillong               1073
Talcher                4014
Thiruvananthapuram     3805
Visakhapatnam          2382
dtype: int64


/tmp/ipython-input-2206697564.py:2: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



# **Inference:**


---
The number of missing values varies widely across cities, showing that data completeness is not uniform.
* Some cities—such as **Mumbai, Ahmedabad, Jorapokhar, Gurugram, Lucknow, and Patna**—have a high number of missing entries, indicating inconsistent pollutant monitoring or long periods of missing recordings.
* In contrast, cities like **Chandigarh, Kochi, Aizawl, Coimbatore, and Ernakulam** have very few missing values, meaning their data is more complete and reliable.
* Most other cities fall somewhere in between, with moderate levels of missing data.
* Overall, the dataset covers all cities consistently in terms of dates, but the quality of pollutant measurements differs from city to city, and cities with large gaps.

In [173]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

In [174]:
# Make a copy of original data BEFORE any cleaning
df_raw = df.copy()

# **Handling Missing Values**

**Xylene** was removed from the dataset because it had more than 50% missing values across cities. With such a large portion of data unavailable, any attempt to fill in the gaps would be mostly guesswork and could introduce bias into the analysis. Dropping the Xylene column helps maintain the reliability and accuracy of the remaining dataset for further processing and modeling.

In [175]:
# 1) Drop Xylene  because more than 50 % data is missing
#Why drop Xylene now?
#When >50% of values are missing, any imputation will mainly be guesswork and will bias downstream results
df = df.drop(columns=["Xylene"])

# **Methods Used for handling missing values :**


---


To ensure the dataset remained accurate and suitable for time-series analysis, **different imputation strategies were applied based on the behavior and characteristics of each pollutant.** Instead of using simple mean or median filling—which can distort trends in time-dependent data—the methods were chosen to preserve natural temporal patterns, seasonal variation, and chemical consistency.
**Time-based interpolation** was used for pollutants with strong hourly fluctuations, **monthly medians** were applied where longer-term trends were more stable, and chemically related gases were balanced using known **scientific relationships**.

These approaches help maintain realistic pollutant dynamics and produce cleaner, more reliable data for further analysis.

In [176]:
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
df = df.sort_values(["City", "Date"])

cols_to_interp = ["PM2.5", "PM10", "CO"]

def interpolate_city(group):
    group = group.set_index("Date") #temporary index creation
    for col in cols_to_interp:
        group[col] = group[col].interpolate(
            method="time",
            limit=3,          # fill small gaps
            limit_direction="both"
        )
    return group.reset_index()

df = df.groupby("City", group_keys=False).apply(interpolate_city)
print(df[cols_to_interp].isna().sum())


PM2.5     3869
PM10     10368
CO        1238
dtype: int64


/tmp/ipython-input-257547496.py:16: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



In [177]:
# Sort by Date only
df = df.sort_values("Date")

def interpolate_date(df):
    df2 = df.set_index("Date")
    for col in cols_to_interp:
        df2[col] = df2[col].interpolate(
            method="time",
            limit_direction="both"
        )
    return df2.reset_index()

df = interpolate_date(df)
print(df[cols_to_interp].isna().sum())


PM2.5    0
PM10     0
CO       0
dtype: int64


# **Inference :**


---

For key pollutants like **PM2.5, PM10, and CO**, **time-based interpolation** was applied using both **city + date** as the primary method and **date-based interpolation** with a small limit as a fallback. This approach fills gaps by estimating values from nearby timestamps within the same city, preserving the natural temporal flow of pollutant concentrations.

As a result, these columns now have complete and smoothly aligned values without distorting underlying patterns.


In [178]:
df['Month'] = df['Date'].dt.month

# Columns to fill with monthly median
cols_to_fill = ["SO2", "NH3", "Benzene", "Toluene","O3"]

for col in cols_to_fill:
    if col in df.columns:
        df[col] = df[col].round(2)
        df[col] = df.groupby(['Month'])[col].transform(lambda x: x.fillna(x.median()))
        print(f"Monthly median filled: {col}")

# Check remaining missing
print(df[cols_to_fill].isna().sum())

Monthly median filled: SO2
Monthly median filled: NH3
Monthly median filled: Benzene
Monthly median filled: Toluene
Monthly median filled: O3
SO2        0
NH3        0
Benzene    0
Toluene    0
O3         0
dtype: int64


# **Inference:**


---
For pollutants such as **SO₂, NH₃, Benzene, Toluene, and O₃**,**monthly median** imputation was used because these pollutants often exhibit longer-term variability rather than rapid hourly fluctuations.

 Monthly medians capture seasonal behavior and typical concentration levels for that month, making them more reliable than standard imputation techniques.



In [192]:


# List of columns
cols = ['NO', 'NO2', 'NOx']

# Single-pass loop
for col in cols:
    # Fill using chemical relationship --> NO + NO2 = NOx
    if col == 'NO':
        df[col] = df[col].combine_first(df['NOx'] - df['NO2'])
    elif col == 'NO2':
        df[col] = df[col].combine_first(df['NOx'] - df['NO'])
    elif col == 'NOx':
        df[col] = df[col].combine_first(df['NO'] + df['NO2'])

    df[col] = df[col].round(2)
    # Fill any remaining missing with monthly median
    df[col] = df.groupby('Month')[col].transform(lambda x: x.fillna(x.median()))

# Check missing values
print(df[cols].isna().sum())


NO     0
NO2    0
NOx    0
dtype: int64


In [201]:
(df[['NO', 'NO2', 'NOx']] < 0).sum()


,0
NO,0
NO2,0
NOx,0


In [199]:
No_pollutants = ['NO', 'NO2', 'NOx']  # Add others if needed

for col in No_pollutants:
    # Step 1: Identify negative values and convert them to NaN
    df.loc[df[col] < 0, col] = np.nan

    # Step 2: Fill those NaNs using monthly median
    # Changed .apply() to .transform() to ensure index alignment
    df[col] = df.groupby(['Month'])[col].transform(lambda x: x.fillna(x.median()))

In [200]:
df.isnull().sum()


,0
Date,0
City,0
PM2.5,0
PM10,0
NO,0
NO2,0
NOx,0
NH3,0
CO,0
SO2,0


In [194]:
(df[['NO', 'NO2', 'NOx']] < 0).sum()

,0
NO,0
NO2,0
NOx,0


# **Inference:**


---

For nitrogen compounds **(NO, NO₂, and NOx)**, a chemistry-based balancing method was used, leveraging the known relationship **NO + NO₂ = NOx**. This ensures scientific consistency in the dataset and prevents unrealistic combinations of nitrogen oxide values.

**Research Paper Reference** :

https://www.sciencedirect.com/science/article/abs/pii/S1352231022006276

https://envs.au.dk/en/research-areas/air-pollution-emissions-and-effects/the-monitoring-program/measuring-methods/nitrogen-oxides-nox-no

In [186]:

# CPCB breakpoints

pm25_breakpoints = [
    (0,   30,   0,   50),
    (31,  60,  51,  100),
    (61,  90, 101,  200),
    (91, 120, 201,  300),
    (121,250, 301,  400),
    (251,500, 401,  500),  # extended severe
]

pm10_breakpoints = [
    (0,    50,  0,   50),
    (51,  100, 51,  100),
    (101, 250,101,  200),
    (251, 350,201,  300),
    (351, 430,301,  400),
    (431,1000,401,  500),  # extended severe
]


def sub_index(conc, bps):
    """
    conc: integer (after rounding)
    bps: list of breakpoints
    """
    if pd.isna(conc):
        return None

    for low, high, i_low, i_high in bps:
        if low <= conc <= high:
            return (i_high - i_low) * (conc - low) / (high - low) + i_low

    return None  # for impossible out-of-range values


# ---------------------------------------
# 3. ROUND concentrations BEFORE applying breakpoints (CPCB rule)
"""✔ Document 1 – CPCB Final AQI Report

https://cpcb.nic.in/displaypdf.php?id=bmF0aW9uYWwtYWlyLXF1YWxpdHktaW5kZXgvRklOQUwtUkVQT1JUX0FRSV8ucGRm
Page 17 → Section 4.1: “Rounding of data”
It states:
“Concentrations should be rounded to the nearest integer before computing sub-index.”"""
# ---------------------------------------

df['PM2.5_round'] = df['PM2.5'].round().astype("Int64")
df['PM10_round']  = df['PM10'].round().astype("Int64")


# ---------------------------------------
# 4. Compute sub-indices
# ---------------------------------------

df['PM2.5_sub'] = df['PM2.5_round'].apply(lambda x: sub_index(x, pm25_breakpoints))
df['PM10_sub']  = df['PM10_round'].apply(lambda x: sub_index(x, pm10_breakpoints))


# ---------------------------------------
# 5. AQI = max of available sub-indices
# ---------------------------------------

df['AQI_recalc'] = df[['PM2.5_sub', 'PM10_sub']].max(axis=1).round().astype('Int64')


# ---------------------------------------
# 6. Assign AQI Bucket (CPCB categories)
# ---------------------------------------

def aqi_bucket_cpcb(aqi):
    if pd.isna(aqi):
        return None
    a = float(aqi)
    if a <= 50:
        return "Good"
    elif a <= 100:
        return "Satisfactory"
    elif a <= 200:
        return "Moderate"
    elif a <= 300:
        return "Poor"
    elif a <= 400:
        return "Very Poor"
    else:
        return "Severe"

df['AQI_Bucket_recalc'] = df['AQI_recalc'].apply(aqi_bucket_cpcb)




In [187]:
#dropping temporary columns
df = df.drop(columns=[
    'PM2.5_round',
    'PM10_round',
    'PM2.5_sub',
    'PM10_sub'
])


# **Inference:**


---

* Recalculating **AQI and AQI_Bucket** from the cleaned pollutant data is a more reliable approach than trying to fill their missing values directly.
* AQI is derived from multiple pollutants, and its accuracy depends on the correctness of each pollutant’s concentration.

 **Why not fill directly?**
* If AQI were simply imputed with mean or median values, it would break the link between AQI and the underlying pollutants, leading to inconsistent or misleading results.
* By recomputing AQI after properly handling missing pollutant values, the index remains scientifically valid and fully aligned with the updated dataset.

 This ensures that the AQI categories (AQI_Bucket) also reflect real pollution levels rather than approximations, providing a cleaner and more trustworthy dataset for analysis and modeling.

 **Research Paper Reference:**

 https://mausam.imd.gov.in/imd_latest/contents/pdf/emrc_sop.pdf

https://www.cpcb.gov.in/displaypdf.php?id=bmF0aW9uYWwtYWlyLXF1YWxpdHktaW5kZXgvRklOQUwtUkVQT1JUX0FRSV8ucGRm


In [195]:
df.isnull().sum()


,0
Date,0
City,0
PM2.5,0
PM10,0
NO,63
NO2,558
NOx,0
NH3,0
CO,0
SO2,0
